In [4]:
from planet4 import markings, dbscan, io, fnotching

In [4]:
from nbtools.logging import setup_live_logging
import logging

In [ ]:
setup_live_logging('planet4', logging.DEBUG)

# Data to cluster

In [6]:
from planet4 import region_data, io

In [7]:
db = io.DBManager()

In [8]:
obsids = db.image_names

In [9]:
db.dbname

'/Users/klay6683/Dropbox/data/planet4/2017-01-01_planet_four_classifications_queryable_cleaned_seasons2and3.h5'

In [11]:
len(obsids)

221

# Support functions

In [12]:
from planet4.catalog_production import process_obsid_parallel

In [18]:
savedir="catalog_1.0b3"

In [19]:
def create_parallel_args(obsids):
    bucket = []
    for obsid in obsids:
        pm = io.PathManager(obsid=obsid, datapath=folder)
        path = pm.obsid_results_savefolder / obsid
        if not path.exists():
            bucket.append(obsid)
    return bucket

In [20]:
bucket = create_parallel_args(obsids)

In [16]:
from nbtools import execute_in_parallel

In [22]:
todo = [(i, folder) for i in bucket]

In [23]:
todo[0]

('ESP_012079_0945', 'catalog_1.0b3')

In [24]:
process_obsid_parallel(todo[0])

'ESP_012079_0945'

In [25]:
todo[1:2]

[('ESP_011697_0980', 'catalog_1.0b3')]

In [28]:
results = execute_in_parallel(process_obsid_parallel, todo[2:])

IntProgress(value=0, max=218)

In [29]:
for result in results:
    print(result)

ESP_021455_0935
ESP_012008_0975
ESP_012436_0980
ESP_020347_0930
ESP_020519_0935
ESP_011931_0945
ESP_011960_0925
ESP_012291_0980
ESP_012077_0930
ESP_011341_0980
ESP_011711_0930
ESP_021495_0935
ESP_020742_0925
ESP_012063_0945
ESP_020357_0950
ESP_021642_0980
ESP_011350_0945
ESP_012884_0935
ESP_011900_0985
ESP_021494_0945
ESP_021454_0925
ESP_020339_0985
ESP_020954_0935
ESP_012499_0930
ESP_011407_0945
ESP_020476_0950
ESP_020748_0985
ESP_020782_0945
ESP_021526_0985
ESP_012251_0935
ESP_021491_0950
ESP_020677_0980
ESP_020779_0950
ESP_012344_0950
ESP_021587_0980
ESP_012227_0980
ESP_011447_0950
ESP_012076_0945
ESP_012817_0925
ESP_020598_0935
ESP_012693_0950
ESP_012256_0985
ESP_020376_0980
ESP_020783_0950
ESP_012691_0985
ESP_021520_0925
ESP_012322_0985
ESP_020322_0930
ESP_012648_0930
ESP_011296_0975
ESP_022273_0950
ESP_012643_0945
ESP_011680_1055
ESP_021903_0930
ESP_020716_0945
ESP_020214_0935
ESP_012316_0925
ESP_012704_0850
ESP_020401_0950
ESP_012393_0965
ESP_011565_0930
ESP_012860_0980
ESP_0128

# Debugging

In [ ]:
from planet4.fnotching import get_clusters_in_path

def find_bad_obsid(obsid, savedir):
    pm = io.PathManager(obsid=obsid, datapath=savedir)
    paths = pm.get_obsid_paths('L1A')
    if len(paths) == 0:
        logger.warning("No paths to fnotch found for %s", obsid)
    fans = []
    blotches = []
    for path in paths:
        f, b = get_clusters_in_path(path)
        fans.append(f)
        blotches.append(b)
    fans = pd.concat(fans, ignore_index=True)

In [24]:
folder

'catalog_1.0b2'

In [ ]:
for obsid in obsids:
    print(obsid)
    find_bad_obsid(obsid, folder)

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
from planet4.io import get_subframe

In [ ]:
obsid = 'ESP_022273_0950'

In [ ]:
data = db.get_image_name_markings(obsid)[['image_id', 'image_url']]

In [ ]:
data.set_index('image_id', drop=True, inplace=True)

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
def show_subframe(image_id):
    img = get_subframe(data.loc[data.index[0]].values[0])
    return hv.Raster(img)

In [ ]:
%output size=150

In [ ]:
%%opts Raster [xaxis=None, yaxis=None]
show_subframe(data.index[0])+show_subframe(data.index[1])

In [ ]:
dmap = hv.DynamicMap(show_subframe, kdims=['image_id'])
dmap

In [ ]:
dmap.redim.values(image_id=data.index.values)

In [ ]:
xvals = np.linspace(-4,0,202)
yvals = np.linspace(4,0,202)
xs,ys = np.meshgrid(xvals, yvals)

def waves_image(alpha, beta):
    return hv.Image(np.sin(((ys/alpha)**alpha+beta)*xs))

waves_image(0,0) + waves_image(0,4)

In [ ]:
dmap = hv.DynamicMap(waves_image, kdims=['alpha', 'beta'])
dmap

In [ ]:
dmap[0,1] + dmap.select(alpha=1, beta=2)

In [ ]:
dmap.redim.range(alpha=(0,5.0), beta=(1,5.0))

In [ ]:
hv.__version__

In [ ]:
for imgid in image_ids:
    do_cluster_ids(imgid, "without_large_fan_fnotch_on_p4_coords" )

In [ ]:
obsid = 'ESP_020930_0980'
savedir = 'fnotching_per_obsid'

In [ ]:
dbscanner = dbscan.DBScanner(savedir=savedir)

In [ ]:
dbscanner.cluster_image_name(obsid)

In [ ]:
pm.final_blotchfile

In [ ]:
fnotching.fnotch_obsid(obsid, savedir=savedir)

In [ ]:
fnotching.apply_cut_obsid(obsid, savedir=savedir)

In [ ]:
data = region_data.Inca.season2 + region_data.Inca.season3

In [ ]:
do_clustering(ithaca[0])

In [ ]:
ithaca[0]

In [ ]:
from nbtools import execute_in_parallel

In [ ]:
result = execute_in_parallel(do_clustering, data)

# Creating obsid files

In [96]:
from planet4.fnotching import get_clusters_in_path

In [99]:
pm = io.PathManager(obsid=roi.seasons[0], datapath=folder)

NameError: name 'roi' is not defined

In [98]:
pm.obsid_final_blotches_path

PosixPath('/Users/klay6683/Dropbox/data/planet4/catalog_1.0b2/ESP_011408_0930_blotches.csv')

In [147]:
from planet4.catalog_production import create_roi_file

In [35]:
savedir

'catalog_1.0b3'

In [148]:
create_roi_file(obsids, 'catalog', savedir)

100%|██████████| 221/221 [03:13<00:00,  1.14it/s]


In [118]:
pm = io.PathManager(obsid=obsids[0], datapath=savedir)

In [119]:
folders = pm.get_obsid_paths('L1C')

In [132]:
from planet4.catalog_production import read_csvfiles_into_lists_of_frames, concat_bucket_lists

In [ ]:
bucket.items()

In [102]:
roi.available_seasons

['season2', 'season3']

In [125]:
folder

'catalog_1.0b2'

In [104]:
def create_roi_file_with_seasons(roi, datapath, copy_to_analysis=True):
    Fans = []
    Blotches = []
    for season in roi.available_seasons:
        print(season)
        # keep lower case marking objects to add season column later
        fans = []
        blotches = []
        for obsid in getattr(roi, season):
            print(obsid)
            pm = io.PathManager(obsid=obsid, datapath=datapath)
            fans.append(pm.final_fandf)
            blotches.append(pm.final_blotchdf)

        fans = pd.concat(fans, ignore_index=True)
        fans['season'] = season
        Fans.append(fans)
    
        blotches = pd.concat(blotches, ignore_index=True)
        blotches['season'] = season
        Blotches.append(blotches)
        
    Fans = pd.concat(Fans, ignore_index=True)
    Blotches = pd.concat(Blotches, ignore_index=True)
    savedir = pm.path_so_far.parent
    for marks in ['Fans', 'Blotches']:
        savename = f"{roi.name}_{pm.L1C_folder}_{marks}.csv"
        savepath = savedir / savename
        obj = eval(marks)  # get handle to the fan or blotch object
        obj = obj.dropna(how='all', axis=1)
        obj.to_csv(savepath, index=False)
        if copy_to_analysis is True:
            savefolder = io.analysis_folder() / folder / f"{roi.name.lower()}"
            savefolder.mkdir(exist_ok=True)
            savepath = savefolder / savename
            obj.to_csv(savepath, index=False)

In [132]:
create_roi_file_with_seasons(roi, folder)

season2
ESP_011460_0980
ESP_011526_0980
ESP_011737_0980
ESP_012515_0980
ESP_012594_0980
ESP_012805_0980
ESP_012871_0980
season3
ESP_022510_0980
ESP_021587_0980
ESP_021574_0980
ESP_021521_0980
ESP_021508_0980
ESP_020941_0980
ESP_020875_0980
ESP_020374_0980
ESP_020163_0980


In [ ]:
def create_obsid_result_files(obsid, datapath=None):
#     logger.debug("Working on %s", obsid)
    pm = io.PathManager(obsid=obsid, datapath=datapath)
    # the pm.get_obsid_paths method searches for existing image_id folders inside each
    # obsid folder
    folders = list(pm.get_obsid_paths(pm.L1C_folder))
    blotches = []
    fans = []
    for folder in folders:
        pm.id = fnotching.get_id_from_path(folder)
        if pm.final_blotchfile.exists() and pm.final_blotchfile.lstat().st_size > 1:
            blotches.append(pm.final_blotchdf)
        if pm.final_fanfile.exists() and pm.final_fanfile.lstat().st_size > 1:
            fans.append(pm.final_fandf)

    if len(blotches) > 0:
        blotches = pd.concat(blotches, ignore_index=True)
        blotches.dropna(how='all', axis=1, inplace=True)
        blotches.to_csv(pm.obsid_final_blotches_path, index=False)
        print("Created", pm.obsid_final_blotches_path)

    if len(fans) > 0:
        fans = pd.concat(fans, ignore_index=True)
        fans.dropna(how='all', axis=1, inplace=True)
        fans.to_csv(pm.obsid_final_fans_path, index=False)
        print("Created", pm.obsid_final_fans_path)

In [ ]:
for obsid in data:
    create_obsid_result_files(obsid, datapath='for_chase')

In [ ]:
region = getattr(region_data, name)
for season_no in [2, 3]:
    season = str(season_no)
    bucket = []
    for obsid in getattr(region, f"season{season}"):
        pm = io.PathManager(obsid=obsid)
        df = pd.read_csv(pm.obsid_final_blotches_path)
        df['obsid'] = obsid
        bucket.append(df)

    df = pd.concat(bucket, ignore_index=True)
    df.to_csv(io.analysis_folder() / f"p4_catalog/{name}_season{season}_blotches.csv",
              index=False)

In [ ]:
store_folder = io.analysis_folder() / 'for_chase'
store_folder.mkdir(exist_ok=True)

In [ ]:
name = 'inca'

In [ ]:
p = Path('/Users/klay6683/Dropbox/data/planet4/for_chase/' + name)
for kind in ['blotches', 'fans']:
    bucket = []
    filepaths = p.glob(f'*_{kind}.csv')
    for filepath in filepaths:
        obsid = filepath.name[:15]
        df = pd.read_csv(filepath)
        df['obsid'] = obsid
        bucket.append(df)
    df = pd.concat(bucket, ignore_index=True)
    savedir = store_folder / f'{name}'
    savedir.mkdir(exist_ok=True)
    savepath = savedir / f"{name}_{kind}.csv"
    print(savepath)
    df.to_csv(savepath, index=False)
        